In [ ]:
class Page:
    """
    A wikipedia page
    """
    def __init__(self, title):
        self.title = title
        self.text = None # query to get text
        self.links = [None] # manipulate text to get links
        self.backlinks = None # access the 'links to this page' in the tools on the sidebar

    def __repr__(self):
        return self.title

    def __str__(self):
        return self.title

    def __eq__(self, other):
        return self.title == other.title

    def __hash__(self):
        return hash(self.title)
    

In [ ]:
class PageWalker:
    """
    A page walker, not really implemented but will look a lot like this
    """
    def __init__(self, start_page, depth):
        self.start_page = start_page
        self.pages = [start_page]
        

    def __iter__(self):
        return self

    def __next__(self):
        if len(self.pages) == 0:
            raise StopIteration
        page = self.pages.pop()
        for link in page.links:
            if link not in self.visited:
                self.visited.add(link)
                self.pages.append(link)
        return page